## Preprocessing for School Location Database


In [207]:
# make pandas show all rows when printing dataframe
pd.set_option('display.max_rows', None)

In [208]:
import os
import pandas as pd
from shapely.geometry import Point

# Directory containing the school CSV files
schools_dir = '../data/landing/schools'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(schools_dir) if f.endswith('.csv')]

# Load each CSV and collect their columns
schemas = {}
for file in csv_files:
    try:
        df = pd.read_csv(os.path.join(schools_dir, file), nrows=0, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(os.path.join(schools_dir, file), nrows=0, encoding='latin1')
    schemas[file] = set(df.columns)

# Display the columns for each file
for file, cols in schemas.items():
    print(f"{file}: {sorted(cols)}")

# Check if all schemas match
all_schemas = list(schemas.values())
schemas_match = all(s == all_schemas[0] for s in all_schemas)
print(f"\nSchemas match: {schemas_match}")

school_locations_2023.csv: ['Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town', 'School_Name', 'School_No', 'School_Status', 'School_Type', 'X', 'Y']
school_locations_2025.csv: ['Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Area', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'LGA_TYPE', 'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town', 'Region', 'School_Name', 'School_No', 'School_Status', 'School_Type', 'X', 'Y']
school_locations_2024.csv: ['AREA_Name', 'Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Postal_Address_Line_1', 'Postal_Address_Lin

In [209]:
# Define the standardized schema based on common columns across all years
standard_columns = [
    'Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town',
    'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name',
    'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town',
    'School_Name', 'School_No', 'School_Type', 'X', 'Y',
    # Additional columns that exist in some years
    'Area', 'LGA_TYPE', 'Region', 'School_Status'
]

print("Standardized schema:")
for col in standard_columns:
    print(f"  - {col}")


Standardized schema:
  - Address_Line_1
  - Address_Line_2
  - Address_Postcode
  - Address_State
  - Address_Town
  - Education_Sector
  - Entity_Type
  - Full_Phone_No
  - LGA_ID
  - LGA_Name
  - Postal_Address_Line_1
  - Postal_Address_Line_2
  - Postal_Postcode
  - Postal_State
  - Postal_Town
  - School_Name
  - School_No
  - School_Type
  - X
  - Y
  - Area
  - LGA_TYPE
  - Region
  - School_Status


In [210]:
# Function to standardize column names and add missing columns
def standardize_school_dataframe(df, year):
    """
    Standardize a school dataframe to have consistent columns.
    
    Parameters:
    df: pandas DataFrame
    year: string indicating the year (2023, 2024, or 2025)
    
    Returns:
    pandas DataFrame with standardized columns
    """
    # Create a copy to avoid modifying the original
    df_std = df.copy()
    
    # Handle column name variations
    column_mapping = {
        'AREA_Name': 'Area',  # 2024 has AREA_Name instead of Area
        'Region_Name': 'Region'  # 2024 has Region_Name instead of Region
    }
    
    # Rename columns
    df_std = df_std.rename(columns=column_mapping)
    
    # Add missing columns with NaN values
    for col in standard_columns:
        if col not in df_std.columns:
            df_std[col] = None
    
    # Reorder columns to match standard schema
    df_std = df_std[standard_columns]
    
    # Add year column to indicate when school was established
    df_std['establishment_year'] = year
    
    return df_std

# Test the function with a small sample
print("Testing standardization function...")
for file in csv_files:
    year = file.split('_')[-1].split('.')[0]  # Extract year from filename
    print(f"\nProcessing {file} (year: {year})")
    
    # Load a small sample to test
    try:
        df_sample = pd.read_csv(os.path.join(schools_dir, file), nrows=5, encoding='utf-8')
    except UnicodeDecodeError:
        df_sample = pd.read_csv(os.path.join(schools_dir, file), nrows=5, encoding='latin1')
    
    # Standardize
    df_std = standardize_school_dataframe(df_sample, year)
    
    print(f"Original columns: {len(df_sample.columns)}")
    print(f"Standardized columns: {len(df_std.columns)}")
    print(f"Missing columns added: {[col for col in standard_columns if col not in df_sample.columns]}")


Testing standardization function...

Processing school_locations_2023.csv (year: 2023)
Original columns: 21
Standardized columns: 25
Missing columns added: ['Area', 'LGA_TYPE', 'Region']

Processing school_locations_2025.csv (year: 2025)
Original columns: 24
Standardized columns: 25
Missing columns added: []

Processing school_locations_2024.csv (year: 2024)
Original columns: 22
Standardized columns: 25
Missing columns added: ['Area', 'LGA_TYPE', 'Region', 'School_Status']


In [211]:
# Load and standardize all school datasets
print("Loading and standardizing all school datasets...")
standardized_dfs = []

for file in csv_files:
    year = file.split('_')[-1].split('.')[0]  # Extract year from filename
    print(f"\nProcessing {file} (year: {year})")
    
    # Load the full dataset
    try:
        df = pd.read_csv(os.path.join(schools_dir, file), encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(os.path.join(schools_dir, file), encoding='latin1')
    
    print(f"  Loaded {len(df)} schools")
    
    # Standardize the dataframe
    df_std = standardize_school_dataframe(df, year)
    standardized_dfs.append(df_std)
    
    print(f"  Standardized to {len(df_std.columns)} columns")

# Combine all standardized dataframes
print(f"\nCombining {len(standardized_dfs)} datasets...")
combined_schools = pd.concat(standardized_dfs, ignore_index=True)

print(f"Combined dataset shape: {combined_schools.shape}")
print(f"Total schools: {len(combined_schools)}")
print(f"\nEstablishment year distribution:")
print(combined_schools['establishment_year'].value_counts().sort_index())


Loading and standardizing all school datasets...

Processing school_locations_2023.csv (year: 2023)
  Loaded 2302 schools
  Standardized to 25 columns

Processing school_locations_2025.csv (year: 2025)
  Loaded 2301 schools
  Standardized to 25 columns

Processing school_locations_2024.csv (year: 2024)
  Loaded 2294 schools
  Standardized to 25 columns

Combining 3 datasets...
Combined dataset shape: (6897, 25)
Total schools: 6897

Establishment year distribution:
establishment_year
2023    2302
2024    2294
2025    2301
Name: count, dtype: int64


In [212]:
# select School_Name, School_No, School_Type, School_Status, establishment_year, X, Y
combined_schools = combined_schools[['School_Name', 'Education_Sector', 'School_Type', 'School_Status', 'establishment_year', 'X', 'Y']]

In [213]:
# round the X, Y columns to 4 decimal places
combined_schools['X'] = combined_schools['X'].round(1)
combined_schools['Y'] = combined_schools['Y'].round(1)


In [214]:
# convert the X, Y columns to a Point object from shapely 
combined_schools['coordinates'] = combined_schools.apply(lambda row: Point(row['X'], row['Y']), axis=1)

In [215]:
combined_schools['establishment_year'].value_counts()

establishment_year
2023    2302
2025    2301
2024    2294
Name: count, dtype: int64

In [216]:
# remove duplicates keeping the occurence with the earliest establishment_year by X, Y
combined_schools = combined_schools.sort_values(by="establishment_year").drop_duplicates(subset=["X", "Y"], keep='first')

# sort by school_name
combined_schools = combined_schools.sort_values(by='School_Name')

In [217]:
combined_schools['School_Type'].value_counts()

School_Type
Primary      332
Secondary     92
Pri/Sec       60
Special        5
Language       2
Camp           1
Name: count, dtype: int64

In [218]:
# remove 'School_Type' where it is 'Language', 'Camp' 
combined_schools = combined_schools[~combined_schools['School_Type'].isin(['Language', 'Camp'])]

combined_schools['School_Type'].value_counts()

School_Type
Primary      332
Secondary     92
Pri/Sec       60
Special        5
Name: count, dtype: int64

In [219]:
combined_schools['School_Status'].value_counts()

School_Status
O    489
Name: count, dtype: int64

In [220]:
# remove 'School_Status' where it is 'Closed'
combined_schools = combined_schools[combined_schools['School_Status'] != 'C']

In [221]:
# drop School_Status column
combined_schools = combined_schools.drop(columns=['School_Status'])



In [ ]:
# convert column names to lower casing
combined_schools.columns = combined_schools.columns.str.lower()

combined_schools.head(5)

,school_name,education_sector,school_type,establishment_year,x,y,coordinates
1205,Aireys Inlet Primary School,Government,Primary,2023,144.1,-38.5,POINT (144.1 -38.5)
0,Alberton Primary School,Government,Primary,2023,146.7,-38.6,POINT (146.7 -38.6)
1295,Alexandra Secondary College,Government,Secondary,2023,145.7,-37.2,POINT (145.7 -37.2)
1542,Alkira Secondary College,Government,Secondary,2023,145.3,-38.1,POINT (145.3 -38.1)
1,Allansford and District Primary School,Government,Primary,2023,142.6,-38.4,POINT (142.6 -38.4)
1235,Alvie Primary School,Government,Primary,2023,143.5,-38.2,POINT (143.5 -38.2)
282,Amphitheatre Primary School,Government,Primary,2023,143.4,-37.2,POINT (143.4 -37.2)
323,Anakie Primary School,Government,Primary,2023,144.3,-37.9,POINT (144.3 -37.9)
660,Anglesea Primary School,Government,Primary,2023,144.2,-38.4,POINT (144.2 -38.4)
1236,Apollo Bay P-12 College,Government,Pri/Sec,2023,143.7,-38.8,POINT (143.7 -38.8)


In [223]:
# check for duplicate school_name
duplicate_schools = combined_schools[combined_schools.duplicated(subset=['school_name'])]

duplicate_schools


,school_name,education_sector,school_type,establishment_year,x,y,coordinates
1634,Sacred Heart School,Catholic,Primary,2023,145.2,-36.4,POINT (145.2 -36.4)
1638,St Brigid's School,Catholic,Primary,2023,144.6,-37.5,POINT (144.6 -37.5)
1640,St Joseph's School,Catholic,Primary,2023,146.6,-36.1,POINT (146.6 -36.1)
1763,St Joseph's School,Catholic,Primary,2023,145.2,-36.8,POINT (145.2 -36.8)
1810,St Joseph's School,Catholic,Primary,2023,144.7,-38.3,POINT (144.7 -38.3)
1724,St Joseph's School,Catholic,Primary,2023,143.9,-35.7,POINT (143.9 -35.7)
1649,St Joseph's School,Catholic,Primary,2023,142.3,-37.9,POINT (142.3 -37.9)
1695,St Joseph's School,Catholic,Primary,2023,146.7,-36.4,POINT (146.7 -36.4)
1710,St Joseph's School,Catholic,Primary,2023,144.3,-36.8,POINT (144.3 -36.8)
1696,St Joseph's School,Catholic,Primary,2023,141.7,-37.6,POINT (141.7 -37.6)


In [224]:
# group duplicate_schools by school_name, education_sector, school_type, establishment_year and count the number of occurences
duplicate_schools_grouped = duplicate_schools.groupby(['school_name', 'education_sector', 'school_type', 'establishment_year']).size().reset_index(name='count')

# get the school_name from duplicate_schools_grouped where count is greater than 1
duplicate_school_names = duplicate_schools_grouped[duplicate_schools_grouped['count'] > 1]['school_name']

duplicate_school_names

2     St Joseph's School
3       St Mary's School
5    St Patrick's School
Name: school_name, dtype: object

In [225]:
combined_schools_unique = combined_schools[~combined_schools['school_name'].isin(duplicate_school_names)]

combined_schools_unique[combined_schools_unique['school_name'].duplicated()]

,school_name,education_sector,school_type,establishment_year,x,y,coordinates
1634,Sacred Heart School,Catholic,Primary,2023,145.2,-36.4,POINT (145.2 -36.4)
1638,St Brigid's School,Catholic,Primary,2023,144.6,-37.5,POINT (144.6 -37.5)
1700,St Michael's School,Catholic,Primary,2023,144.2,-37.3,POINT (144.2 -37.3)


In [226]:
coburg_special_developmental_school = combined_schools_unique[combined_schools_unique['school_name']=="Coburg Special Developmental School"]


In [227]:
coburg_special_developmental_school

,school_name,education_sector,school_type,establishment_year,x,y,coordinates


In [228]:
import folium

# get the x, y coordinates of coburg_special_developmental_school
x1, y1 = coburg_special_developmental_school['x'].iloc[0], coburg_special_developmental_school['y'].iloc[0]
x2, y2 = coburg_special_developmental_school['x'].iloc[1], coburg_special_developmental_school['y'].iloc[1]


# plot the points (x1, y1) and (x2, y2) on a Folium map by converting to a Point object from shapely
from shapely.geometry import Point

point1 = Point(x1, y1)
point2 = Point(x2, y2)

# plot points on a Folium map
m = folium.Map(location=[y1, x1], zoom_start=15)
folium.Marker(location=[y1, x1], popup='Point 1').add_to(m)
folium.Marker(location=[y2, x2], popup='Point 2').add_to(m)
m


IndexError: single positional indexer is out-of-bounds

In [ ]:
combined_schools_duplicates = combined_schools[combined_schools['school_name'].isin(duplicate_school_names)]



In [ ]:
# Save the combined dataset
output_path = '../data/processed/schools/schools.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

combined_schools.to_csv(output_path, index=False)
